# This file execute a MNIST

In [1]:
from dataset.mnist import load_mnist

In [2]:
# You must wait a minute for the first call
(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)

# outputs data
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)
    

(60000, 784)
(60000,)
(10000, 784)
(10000,)


# Show a MNIST Image using PIL

In [5]:
import sys, os
import numpy as np
from dataset.mnist import load_mnist
import pickle
from PIL import Image

# Show Image function
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    print(type(pil_img))
    pil_img.show()

In [6]:
# Get a MNIST data
(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]

# output property 
print(label)
print(img.shape)

# reshape Image size (28, 28)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)


5
(784,)
(28, 28)
<class 'PIL.Image.Image'>


# Create a Neural Network

### Need the fllowing node
- input layer : 784 (image size)
- output layer : 10 (class size)
- hidding layer1 : 50
- hidding layer2 : 100

※ Hidding layer value is able to any value you can specified

In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)

    y = exp_a / sum_exp_a
    return y

In [14]:
# Makes theree function

def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False)
    
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)

    return network

def predict(network, x):
    # weight
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    # bias
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y

In [25]:
%%time
# execute neaural network
x, t = get_data()
network = init_network()

# debug
# print(x) # Image property
# print(t) # label property
# print(network)

# calculate accuracy
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # Get the most biggest facter of probability index

    if p == t[i]:
        accuracy_cnt += 1


print(f"Accuracy: {str(float(accuracy_cnt) / len(x))}")


Accuracy: 0.9352
Wall time: 2.02 s


# Batch Processing

※Batch: Cohesive input data(A wad of bills image.)

In [23]:
x, _ = get_data()
network = init_network()
w1, w2, w3 = network['W1'], network['W2'], network['W3']

# Shape of x parameter
print(x.shape)
print(x[0].shape)

# Shape of w parameter
print(w1.shape)
print(w2.shape)
print(w3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [26]:
%%time
x, t = get_data()
network = init_network()

batch_size = 100 # number of batch
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print(f"Accuracy: {str(float(accuracy_cnt) / len(x))}")

Accuracy: 0.9352
Wall time: 384 ms


### Topic

#### axis: Specify one dimension as the axis



In [28]:
# For example
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6],
[0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])

y = np.argmax(x, axis=1)
print(y) # Outputs most biggest value in arrange each other

[1 2 1 0]


In [29]:
y = np.array([1, 2, 1, 0])
t = np.array([1, 2, 0, 0])

print(y==t)
print(np.sum(y==t))

[ True  True False  True]
3
